### Methodolody

What I want to do is look at the practice sessions for each race and look at the long runs of each car to find the tyre deg, then apply that and work through lap by lap for the race.

Then obviously the pit stops are whatever has the lowest total race time.

This could be put futher into even looking at other cars and not pitting into traffic, but then we start getting quite complicated.

Really just want to do this as a proof of concept, then look into ML models. May come back and improve to compare to ML models.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))
from DB.models import init_db, Circuit, Season, RacingWeekend, Driver, Session, SessionResult, Lap


## Collect data into df

In [163]:
def create_dataframe():
    # initialize db connection and session
    db_engine, db_session = init_db()

    # query data from the database
    query = db_session.query(
        RacingWeekend.year,
        RacingWeekend.round,
        Circuit.circuit_name,
        Driver.driver_name,
        Driver.driver_short,
        Lap.lap_num,
        Lap.lap_time,
        Lap.tyre,
        Lap.pit,
        Session.session_type
    ).join(RacingWeekend.circuit) \
     .join(RacingWeekend.sessions) \
     .join(Session.laps) \
     .join(Lap.driver) \
     .join(RacingWeekend.season) \
     .all()

    # convert result to list of dicts
    data = []
    for row in query:
        data.append({
            'year': row.year,
            'round': row.round,
            'circuit_name': row.circuit_name,
            'driver_name': row.driver_name,
            'driver_short': row.driver_short,
            'lap_num': row.lap_num,
            'lap_time': row.lap_time,
            'tyre': row.tyre,
            'pit': row.pit,
            'session_type': row.session_type
        })

    # create dataframe
    df = pd.DataFrame(data)

    # encode 'tyre' and 'pit' as categories
    df['tyre'] = df['tyre'].astype('category')
    df['pit'] = df['pit'].astype('category')

    # one-hot encode categorical variables
    df = pd.get_dummies(df, columns=['circuit_name', 'driver_name', 'driver_short', 'tyre', 'session_type'], drop_first=True)

    return df

# create the dataframe and print it
df = create_dataframe()

print(df.columns.tolist())




['year', 'round', 'lap_num', 'lap_time', 'pit', 'circuit_name_Baku', 'circuit_name_Barcelona', 'circuit_name_Budapest', 'circuit_name_Hockenheim', 'circuit_name_Imola', 'circuit_name_Istanbul', 'circuit_name_Jeddah', 'circuit_name_Las Vegas', 'circuit_name_Le Castellet', 'circuit_name_Lusail', 'circuit_name_Marina Bay', 'circuit_name_Melbourne', 'circuit_name_Mexico City', 'circuit_name_Miami', 'circuit_name_Monaco', 'circuit_name_Monte Carlo', 'circuit_name_Montréal', 'circuit_name_Monza', 'circuit_name_Mugello', 'circuit_name_Nürburgring', 'circuit_name_Portimão', 'circuit_name_Sakhir', 'circuit_name_Shanghai', 'circuit_name_Silverstone', 'circuit_name_Singapore', 'circuit_name_Sochi', 'circuit_name_Spa-Francorchamps', 'circuit_name_Spielberg', 'circuit_name_Suzuka', 'circuit_name_São Paulo', 'circuit_name_Yas Island', 'circuit_name_Zandvoort', 'driver_name_Alexander Albon', 'driver_name_Andrea Kimi Antonelli', 'driver_name_Antonio Giovinazzi', 'driver_name_Arthur Leclerc', 'driver_n